# Correlating Returns

In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd
from newsapi.newsapi_client import NewsApiClient
from datetime import date, datetime, timedelta
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [2]:
# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWSAPI_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [37]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = '1D'

# Get current date and the date from one month ago
current_date = date.today()
past_date = date.today() - timedelta(weeks=4)

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=current_date,
    end=past_date,
    after=None,
    until=None,
).df

df.head()

AAPL                                   
                             open    high      low   close    volume
2019-11-05 00:00:00-05:00  257.05  258.19  256.320  257.15  17738372
2019-11-06 00:00:00-05:00  256.84  257.49  255.365  257.24  15328064
2019-11-07 00:00:00-05:00  258.74  260.35  258.110  259.50  22267200
2019-11-08 00:00:00-05:00  258.69  260.44  256.850  260.14  15896858
2019-11-11 00:00:00-05:00  258.30  262.47  258.280  262.20  19401290

In [38]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

df.head()

close
2019-11-05  257.15
2019-11-06  257.24
2019-11-07  259.50
2019-11-08  260.14
2019-11-11  262.20

In [39]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change()
aapl_returns.head()

close
2019-11-05       NaN
2019-11-06  0.000350
2019-11-07  0.008786
2019-11-08  0.002466
2019-11-11  0.007919

In [40]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []
    date = current_date
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > past_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date),
            to=str(date),
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates



In [41]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2020-03-30
retrieving news from: 2020-03-29
retrieving news from: 2020-03-28
retrieving news from: 2020-03-27
retrieving news from: 2020-03-26
retrieving news from: 2020-03-25
retrieving news from: 2020-03-24
retrieving news from: 2020-03-23
retrieving news from: 2020-03-22
retrieving news from: 2020-03-21
retrieving news from: 2020-03-20
retrieving news from: 2020-03-19
retrieving news from: 2020-03-18
retrieving news from: 2020-03-17
retrieving news from: 2020-03-16
retrieving news from: 2020-03-15
retrieving news from: 2020-03-14
retrieving news from: 2020-03-13
retrieving news from: 2020-03-12
retrieving news from: 2020-03-11
retrieving news from: 2020-03-10
retrieving news from: 2020-03-09
retrieving news from: 2020-03-08
retrieving news from: 2020-03-07
retrieving news from: 2020-03-06
retrieving news from: 2020-03-05
retrieving news from: 2020-03-04
retrieving news from: 2020-03-03


In [42]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2020-03-30
retrieving news from: 2020-03-29
retrieving news from: 2020-03-28
retrieving news from: 2020-03-27
retrieving news from: 2020-03-26
retrieving news from: 2020-03-25
retrieving news from: 2020-03-24
retrieving news from: 2020-03-23
retrieving news from: 2020-03-22
retrieving news from: 2020-03-21
retrieving news from: 2020-03-20
retrieving news from: 2020-03-19
retrieving news from: 2020-03-18
retrieving news from: 2020-03-17
retrieving news from: 2020-03-16
retrieving news from: 2020-03-15
retrieving news from: 2020-03-14
retrieving news from: 2020-03-13
retrieving news from: 2020-03-12
retrieving news from: 2020-03-11
retrieving news from: 2020-03-10
retrieving news from: 2020-03-09
retrieving news from: 2020-03-08
retrieving news from: 2020-03-07
retrieving news from: 2020-03-06
retrieving news from: 2020-03-05
retrieving news from: 2020-03-04
retrieving news from: 2020-03-03


In [43]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2020-03-30
retrieving news from: 2020-03-29
retrieving news from: 2020-03-28
retrieving news from: 2020-03-27
retrieving news from: 2020-03-26
retrieving news from: 2020-03-25
retrieving news from: 2020-03-24
retrieving news from: 2020-03-23
retrieving news from: 2020-03-22
retrieving news from: 2020-03-21
retrieving news from: 2020-03-20
retrieving news from: 2020-03-19
retrieving news from: 2020-03-18
retrieving news from: 2020-03-17
retrieving news from: 2020-03-16
retrieving news from: 2020-03-15
retrieving news from: 2020-03-14
retrieving news from: 2020-03-13
retrieving news from: 2020-03-12
retrieving news from: 2020-03-11
retrieving news from: 2020-03-10
retrieving news from: 2020-03-09
retrieving news from: 2020-03-08
retrieving news from: 2020-03-07
retrieving news from: 2020-03-06
retrieving news from: 2020-03-05
retrieving news from: 2020-03-04
retrieving news from: 2020-03-03


In [44]:
# Get fourth topic
iphone_headlines, _ = get_headlines("iphone")

Fetching news about 'iphone'
******************************
retrieving news from: 2020-03-30
retrieving news from: 2020-03-29
retrieving news from: 2020-03-28
retrieving news from: 2020-03-27
retrieving news from: 2020-03-26
retrieving news from: 2020-03-25
retrieving news from: 2020-03-24
retrieving news from: 2020-03-23
retrieving news from: 2020-03-22
retrieving news from: 2020-03-21
retrieving news from: 2020-03-20
retrieving news from: 2020-03-19
retrieving news from: 2020-03-18
retrieving news from: 2020-03-17
retrieving news from: 2020-03-16
retrieving news from: 2020-03-15
retrieving news from: 2020-03-14
retrieving news from: 2020-03-13
retrieving news from: 2020-03-12
retrieving news from: 2020-03-11
retrieving news from: 2020-03-10
retrieving news from: 2020-03-09
retrieving news from: 2020-03-08
retrieving news from: 2020-03-07
retrieving news from: 2020-03-06
retrieving news from: 2020-03-05
retrieving news from: 2020-03-04


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 500 requests over a 24 hour period (250 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [20]:
# Get fifth topic
gold_headlines, _ = get_headlines("gold")

Fetching news about 'gold'
******************************
retrieving news from: 2020-03-30
retrieving news from: 2020-03-29
retrieving news from: 2020-03-28
retrieving news from: 2020-03-27
retrieving news from: 2020-03-26
retrieving news from: 2020-03-25
retrieving news from: 2020-03-24
retrieving news from: 2020-03-23
retrieving news from: 2020-03-22
retrieving news from: 2020-03-21
retrieving news from: 2020-03-20
retrieving news from: 2020-03-19
retrieving news from: 2020-03-18
retrieving news from: 2020-03-17
retrieving news from: 2020-03-16
retrieving news from: 2020-03-15
retrieving news from: 2020-03-14
retrieving news from: 2020-03-13
retrieving news from: 2020-03-12
retrieving news from: 2020-03-11
retrieving news from: 2020-03-10
retrieving news from: 2020-03-09
retrieving news from: 2020-03-08
retrieving news from: 2020-03-07
retrieving news from: 2020-03-06
retrieving news from: 2020-03-05
retrieving news from: 2020-03-04
retrieving news from: 2020-03-03


In [45]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [46]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment



In [67]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)
iphone_avg = headline_sentiment_summarizer_avg(iphone_headlines)
gold_avg = headline_sentiment_summarizer_avg(gold_headlines)


In [70]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
        "iphone_avg": iphone_avg,
        "gold_avg": gold_avg,
    }
)


In [71]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [72]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

display(topic_sentiments)

close  aapl_avg  trade_avg  economy_avg  iphone_avg  gold_avg
2020-03-03 -0.031825  0.120545   0.089980    -0.162220    0.261435  0.130250
2020-03-04  0.046490  0.082130  -0.008500    -0.023815    0.032660  0.021180
2020-03-05 -0.032501 -0.029005  -0.151750    -0.017475    0.082530  0.125445
2020-03-06 -0.013451  0.053715  -0.085335    -0.071220    0.037820  0.112000
2020-03-09 -0.079210  0.119485  -0.006030    -0.267860    0.104155  0.051715
2020-03-10  0.073659  0.122495   0.130400    -0.052925    0.016900  0.222520
2020-03-11 -0.036368  0.097720  -0.042475    -0.141795    0.048175  0.019820
2020-03-12 -0.099165  0.045705  -0.136230    -0.227705    0.061360  0.076135
2020-03-13  0.121210  0.094535  -0.176075    -0.302805    0.133660  0.156840
2020-03-16 -0.131590  0.143135  -0.037960    -0.068840    0.096685 -0.097350
2020-03-17  0.047128 -0.000625  -0.010565    -0.054060    0.014500  0.021985
2020-03-18 -0.015226  0.084250  -0.120680    -0.141440    0.068340 -0.021825
2020-03-19 -0.016948  0.151870   0.103840     0.035890    0.077080  0.019080
2020-03-20 -0.065528 -0.030965  -0.055140    -0.237500    0.104150  0.027035
2020-03-23 -0.023214  0.225195  -0.102410    -0.045060    0.036085 -0.016855
2020-03-24  0.104955  0.218275   0.058625     0.099090    0.221730 -0.002220
2020-03-25 -0.005732  0.001320  -0.053715    -0.093720    0.087475 -0.028740
2020-03-26  0.053022  0.192760  -0.086235    -0.117215    0.148535  0.010565
2020-03-27 -0.042015  0.176805   0.049835    -0.221890    0.129295  0.175025
2020-03-30  0.025672 -0.060730  -0.085855    -0.092660   -0.022280  0.100600

In [66]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()